In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tp-sentinel/Final_merged_tp_sentinel.csv


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.neighbors import KNeighborsRegressor
from sklearn import metrics
from math import sqrt
from sklearn.svm import SVR
from sklearn.linear_model import BayesianRidge
from sklearn.kernel_ridge import KernelRidge
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, mean_absolute_percentage_error
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.model_selection import KFold,RandomizedSearchCV
from sklearn.preprocessing import StandardScaler  
import math
import pandas as pd
from keras import models, layers, optimizers, regularizers
import numpy as np
import random
from sklearn import model_selection, preprocessing
import tensorflow as tf
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from scipy import stats
from tensorflow.keras.callbacks import EarlyStopping
import statsmodels.api as sm
import warnings
warnings.filterwarnings("ignore")
from joblib import dump

2024-03-13 17:06:08.418698: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-13 17:06:08.418792: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-13 17:06:08.569171: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
dataset=pd.read_csv('/kaggle/input/tp-data/tp1_final.csv')
dataset.head()
dataset.dropna(inplace=True)
print(dataset.count())

STATION        351
Date           351
Band1_Mean     351
Band2_Mean     351
Band3_Mean     351
Band4_Mean     351
Band5_Mean     351
Band6_Mean     351
Band7_Mean     351
Band8_Mean     351
Band9_Mean     351
Band10_Mean    351
Band11_Mean    351
Band12_Mean    351
Band13_Mean    351
TEST VALUE     351
dtype: int64


In [4]:
features = ['Band1_Mean','Band2_Mean','Band3_Mean','Band4_Mean','Band5_Mean','Band6_Mean','Band7_Mean','Band8_Mean','Band9_Mean','Band10_Mean','Band11_Mean','Band12_Mean','Band13_Mean']
label = ['TEST VALUE']

X = dataset.loc[:, features].values
y = dataset.loc[:, label].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [5]:
from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Apply StandardScaler to target variable (y)
scaler_y = StandardScaler()
y_train_scaled = scaler_y.fit_transform(y_train.reshape(-1, 1)).ravel()

# Define the kernel
kernel = DotProduct() + WhiteKernel()

# Create GaussianProcessRegressor with the defined kernel
regressor = GaussianProcessRegressor(kernel=kernel)

# Fit the regressor on the training data
regressor.fit(X_train, y_train_scaled)

# Make predictions on the test set
predictions_scaled = regressor.predict(X_test)

# Inverse transform predictions to original scale for evaluation
predictions = scaler_y.inverse_transform(predictions_scaled.reshape(-1, 1)).ravel()

# Evaluate the model
r2 = r2_score(y_test, predictions)
mse = mean_squared_error(y_test, predictions)

print(f'R-squared (R2): {r2}')
print(f'Mean Squared Error: {mse}')

model_filename = 'TP-gaussian.joblib'

# Save the model using joblib
dump(regressor, model_filename)

# Print a message indicating the successful save
print(f'Model saved as {model_filename}')

R-squared (R2): 0.6488239192016922
Mean Squared Error: 0.013173246642966872
Model saved as TP-gaussian.joblib


In [10]:
svr_model = SVR(kernel='rbf')

param_dist = {
    'C': [x for x in range(1, 10001, 1)],
    'epsilon': [x for x in np.arange(0.0001, 1, 0.001)]
}

random_search = RandomizedSearchCV(SVR(), param_distributions=param_dist, n_iter=200, cv=5, scoring='neg_mean_squared_error', random_state=42)
random_search.fit(X_train, y_train)

randomDf = pd.DataFrame(random_search.cv_results_)
best_svr_model = random_search.best_estimator_

y_pred = best_svr_model.predict(X_test)

test_mse = mean_squared_error(y_test, y_pred)
test_r2 = r2_score(y_test, y_pred)
test_mae = mean_absolute_error(y_test, y_pred)
test_mape = mean_absolute_percentage_error(y_test, y_pred) * 100
test_rmse = np.sqrt(test_mse)
bias = best_svr_model.intercept_

print(f"Best Parameters: {random_search.best_params_}")
print(f"Overall R-squared (R2) Score on Test Set: {test_r2 * 100}")
print(f"Overall RMSE on Test Set: {test_rmse}")
print(f"Overall MAE on Test Set: {test_mae}")
print(f"Overall MAPE on Test Set: {test_mape}")
print(f"Overall Bias (Intercept): {bias}")

svr_model_filename = 'TP-svr.joblib'
dump(best_svr_model, svr_model_filename)
print(f'Best SVR model saved as {svr_model_filename}')

Best Parameters: {'epsilon': 0.08710000000000001, 'C': 62}
Overall R-squared (R2) Score on Test Set: 47.78944095343576
Overall RMSE on Test Set: 0.13994684739162985
Overall MAE on Test Set: 0.09018457340647609
Overall MAPE on Test Set: 17.63403659987859
Overall Bias (Intercept): [0.79366812]
Best SVR model saved as TP-svr.joblib


In [7]:
from sklearn.ensemble import AdaBoostRegressor
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.metrics import mean_squared_error
from scipy.stats import uniform, randint
import pandas as pd

# Create AdaBoostRegressor
adaboost_model = AdaBoostRegressor(random_state=42)

# Define the parameter grid
param_dist = {
    'n_estimators': randint(10, 200),
    'learning_rate': uniform(0.01, 1.0),
}

# Create RandomizedSearchCV
random_search = RandomizedSearchCV(adaboost_model, param_distributions=param_dist, n_iter=10, scoring='neg_mean_squared_error', cv=5, random_state=42)

# Fit the model
random_search.fit(X_train, y_train)

# Get the best parameters
best_n_estimators = random_search.best_params_['n_estimators']
best_learning_rate = random_search.best_params_['learning_rate']

# Train the model with the best parameters
best_adaboost_model = AdaBoostRegressor(n_estimators=best_n_estimators, learning_rate=best_learning_rate, random_state=42)
best_adaboost_model.fit(X_train, y_train)

# Make predictions
y_pred = best_adaboost_model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')
r2 = r2_score(y_test, y_pred)

print(f'R-squared (R2): {r2}')

adaboost_model_filename = 'TP-adaboost.joblib'
dump(best_adaboost_model, adaboost_model_filename)
print(f'Best AdaBoostRegressor model saved as {adaboost_model_filename}')

Mean Squared Error: 0.023235604414378178
R-squared (R2): 0.6488239192016922
Best AdaBoostRegressor model saved as TP-adaboost.joblib


In [8]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np


# Create DecisionTreeRegressor
decision_tree_regressor = DecisionTreeRegressor(random_state=42)

# Define the parameter grid
param_dist = {
    'max_depth': np.arange(1, 21),  # Adjust the range based on your preferences
}

# Create RandomizedSearchCV
random_search = RandomizedSearchCV(decision_tree_regressor, param_distributions=param_dist, n_iter=10, scoring='neg_mean_squared_error', cv=5, random_state=42)

# Fit the model
random_search.fit(X_train, y_train)

# Get the best parameter
best_max_depth = random_search.best_params_['max_depth']

# Train the model with the best parameter
best_decision_tree_regressor = DecisionTreeRegressor(max_depth=best_max_depth, random_state=42)
best_decision_tree_regressor.fit(X_train, y_train)

# Make predictions
y_pred = best_decision_tree_regressor.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')
r2 = r2_score(y_test, y_pred)
print(f"R^2 Score on Test Set: {r2}")

decision_tree_model_filename = 'TP-DT.joblib'
dump(best_decision_tree_regressor, decision_tree_model_filename)
print(f'Best DecisionTreeRegressor model saved as {decision_tree_model_filename}')


Mean Squared Error: 0.026773461894631683
R^2 Score on Test Set: 0.28626559022339604
Best DecisionTreeRegressor model saved as TP-DT.joblib


In [11]:
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import pandas as pd
from sklearn.preprocessing import StandardScaler

# Assuming 'dataset' is your DataFrame

# Define features and labels
features = ['Band1_Mean', 'Band2_Mean', 'Band3_Mean', 'Band4_Mean', 'Band5_Mean', 'Band6_Mean', 'Band7_Mean', 'Band8_Mean',
            'Band9_Mean', 'Band10_Mean', 'Band11_Mean', 'Band12_Mean', 'Band13_Mean']
label = ['TEST VALUE']

X = dataset.loc[:, features].values
y = dataset.loc[:, label].values

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the features (important for neural networks)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Create MLPRegressor
mlp_regressor = MLPRegressor(hidden_layer_sizes=(50000, ), activation='relu', solver='adam', max_iter=500, random_state=42)

# Fit the model
mlp_regressor.fit(X_train_scaled, y_train)

# Make predictions
y_pred = mlp_regressor.predict(X_test_scaled)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')
r2 = r2_score(y_test, y_pred)
print(f"R^2 Score on Test Set: {r2}")

mlp_model_filename = 'TP-mlp.joblib'
dump(mlp_regressor, mlp_model_filename)
print(f'MLPRegressor model saved as {mlp_model_filename}')

Mean Squared Error: 0.024225789053944808
R^2 Score on Test Set: 0.3541821629254166
MLPRegressor model saved as TP-mlp.joblib


In [ ]:
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.metrics import mean_squared_error
from scipy.stats import randint
import numpy as np
from sklearn.preprocessing import StandardScaler

# Assuming 'dataset' is your DataFrame

# Define features and labels
features = ['Band1_Mean', 'Band2_Mean', 'Band3_Mean', 'Band4_Mean', 'Band5_Mean', 'Band6_Mean', 'Band7_Mean', 'Band8_Mean',
            'Band9_Mean', 'Band10_Mean', 'Band11_Mean', 'Band12_Mean', 'Band13_Mean']
label = ['TP']

X = dataset.loc[:, features].values
y = dataset.loc[:, label].values

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the features (important for neural networks)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Create MLPRegressor
mlp_regressor = MLPRegressor(max_iter=500, random_state=42)

# Define the parameter grid
param_dist = {
    'hidden_layer_sizes': list(zip(np.arange(10, 70000, 10), np.arange(10, 70000, 10))),  # Adjust the range based on your preferences
}

# Create RandomizedSearchCV
random_search = RandomizedSearchCV(mlp_regressor, param_distributions=param_dist, n_iter=10, scoring='r2', cv=5, random_state=42)

# Fit the model
random_search.fit(X_train_scaled, y_train)

# Get the best parameter
best_hidden_layer_sizes = random_search.best_params_['hidden_layer_sizes']

# Train the model with the best parameter
best_mlp_regressor = MLPRegressor(hidden_layer_sizes=best_hidden_layer_sizes, activation='relu', solver='adam', max_iter=500, random_state=42)
best_mlp_regressor.fit(X_train_scaled, y_train)

# Make predictions
y_pred = best_mlp_regressor.predict(X_test_scaled)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')
r2 = r2_score(y_test, y_pred)
print(f"R^2 Score on Test Set: {r2}")
